In [1]:
import sys
sys.version

'3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:24:02) \n[Clang 11.1.0 ]'

In [2]:
import sqlite3

connection = sqlite3.connect('../data/movies.db')
cursor = connection.cursor()

In [4]:
records = cursor.execute('SELECT * FROM movies;')
for record in records:
    print(record)

('Pulp Fiction', 'Quentin Tarantino', 1994)
('Back to the Future', 'Steven Spielberg', 1985)
('Moonrise Kingdom', 'Wes Anderson', 2012)


In [6]:
cursor.execute('SELECT * FROM movies;')
cursor.fetchall()

[('Pulp Fiction', 'Quentin Tarantino', 1994),
 ('Back to the Future', 'Steven Spielberg', 1985),
 ('Moonrise Kingdom', 'Wes Anderson', 2012)]

In [8]:
cursor.execute('SELECT * FROM movies WHERE title LIKE "B%"')
cursor.fetchall()

[('Back to the Future', 'Steven Spielberg', 1985)]

In [37]:
import sqlalchemy as db
from pathlib import Path

db_path = Path(os.path.abspath('.')).parents[0].absolute().joinpath('data/movies.db')
engine = db.create_engine(f'sqlite:///{db_path}')
connection = engine.connect()

In [42]:
metadata = db.MetaData()
movies = db.Table('movies', metadata, autoload=True, autoload_with=engine)

In [43]:
metadata

MetaData()

In [44]:
movies

Table('movies', MetaData(), Column('title', TEXT(), table=<movies>), Column('director', TEXT(), table=<movies>), Column('year', INTEGER(), table=<movies>), schema=None)

In [45]:
query = db.select([movies])

In [48]:
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()

In [49]:
result_set

[('Pulp Fiction', 'Quentin Tarantino', 1994),
 ('Back to the Future', 'Steven Spielberg', 1985),
 ('Moonrise Kingdom', 'Wes Anderson', 2012)]

In [54]:
query = db.select([movies]).where(movies.columns.director == 'Steven Spielberg')
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
result_set

[('Back to the Future', 'Steven Spielberg', 1985)]

In [80]:
query = movies.insert().values(title='Psycho', director='Alfred Hitchcock', year=1960)
connection.execute(query)

In [81]:
query = db.select(movies)
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
result_set

[('Pulp Fiction', 'Quentin Tarantino', 1994),
 ('Back to the Future', 'Steven Spielberg', 1985),
 ('Moonrise Kingdom', 'Wes Anderson', 2012),
 ('Psycho', 'Alfred Hitchcock', 1960)]

In [82]:
movies.columns.director

Column('director', TEXT(), table=<movies>)

In [86]:
query = db.delete(movies).where(movies.columns.title == 'Psycho')
connection.execute(query)


In [87]:
query = db.select(movies)
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
result_set

[('Pulp Fiction', 'Quentin Tarantino', 1994),
 ('Back to the Future', 'Steven Spielberg', 1985),
 ('Moonrise Kingdom', 'Wes Anderson', 2012)]

In [89]:
metadata.info

{}

In [6]:
import sqlalchemy as db

In [8]:
db.INTEGER

sqlalchemy.sql.sqltypes.INTEGER

In [12]:
from sqlalchemy import Column, Integer, String, Float, ForeignKey
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base

In [13]:
Base = declarative_base()

In [14]:
class Purchase(Base):
    __tablename__ = 'purchases'
    __table_args__ = {'schema': 'landon'}

    order_id = Column(Integer, primary_key=True)
    property_id = Column(Integer)
    property_city = Column(String(250))
    property_state = Column(String(250))
    product_id = Column(Integer)
    product_category = Column(String(250))
    product_name = Column(String(250))
    quantity = Column(Integer)
    product_price = Column(Float)
    order_total = Column(Float)


    def __repr__(self):
        return ''

In [22]:
purchase = Purchase()
purchase.__table__.columns.keys()

['order_id',
 'property_id',
 'property_city',
 'property_state',
 'product_id',
 'product_category',
 'product_name',
 'quantity',
 'product_price',
 'order_total']

In [26]:
s = 'Hello'
f's={repr(s)}'

"s='Hello'"

In [6]:
def format_class_repr_dict(cls_name, **attrs):
    s = ', '.join([f'{k}={repr(v)}' for k, v in attrs.items()])
    return s
    

In [7]:
format_class_repr_dict('Purchase', order_id=1, property_id=2, property_city="UP")


"order_id=1, property_id=2, property_city='UP'"

In [19]:
import psycopg2

conn = psycopg2.connect(host='localhost', user='postgres', password='', port='5432', database='red30')
conn.info

In [30]:
from functools import reduce

In [31]:
def get_column_names(conn, table):
    cur = conn.cursor()
    cur.execute(f"SELECT column_name FROM information_schema.columns WHERE table_schema='public' AND table_name='{table}'")
    cols = reduce(lambda x, y: x+y, cur.fetchall())
    return cols

In [32]:
cur = conn.cursor()
cur.execute('SELECT * FROM sales LIMIT 10;')

In [33]:
get_column_names(conn, 'sales')

('order_num',
 'order_type',
 'cust_name',
 'cust_state',
 'prod_category',
 'prod_number',
 'prod_name',
 'quantity',
 'price',
 'discount',
 'order_total')

In [22]:
from sqlalchemy import create_engine, MetaData, Table, func as sqlfunc
from sqlalchemy.orm import sessionmaker

Session = sessionmaker()
session = Session()

In [38]:
DATABASE = 'red30'
engine = create_engine(f'postgresql://postgres:@localhost:5432/{DATABASE}')

conn = engine.connect()


In [39]:
meta = MetaData(engine)

sales_table = Table('sales', meta, autoload=True, autoload_with=engine)


In [40]:
session.query(sales_table).count()

4977

In [41]:
sales_table.columns.keys()

['order_num',
 'order_type',
 'cust_name',
 'cust_state',
 'prod_category',
 'prod_number',
 'prod_name',
 'quantity',
 'price',
 'discount',
 'order_total']

In [45]:
row_count = session.query(sales_table).count()
last_row = sales_table.select().offset(row_count-1)
session.execute(last_row).first()

(1105909, 'Retail', 'Gussi Strodder', 'New York', 'Drone Kits', 'DK204', 'BYOD-300', 1, 89.0, 0.0, 89.0)

In [48]:
sales_table.c.order_num

Column('order_num', INTEGER(), table=<sales>, primary_key=True, nullable=False)

In [49]:
session.query(sqlfunc.max(sales_table.c.order_num)).scalar()

1105910

In [57]:
delete_statement = sales_table.delete().where(sales_table.c.cust_name == 'Akshay Bhatia')
session.execute(delete_statement)
session.commit()

In [59]:
import psycopg2
DATABASE = 'red30'

conn = psycopg2.connect(
    database=DATABASE,
    host='localhost',
    user='postgres',
    password='',
    port='5432')

In [62]:
cur = conn.cursor()
cur.execute('SELECT max(order_num) FROM sales')
cur.fetchone()[0]

1105911

In [63]:
conn.autocommit

False

In [82]:
import pandas as pd

seq_large = list(range(1, 101))
seq_small = list(range(20, 25))
nlarge = len(seq_large)
nsmall = len(seq_small)

In [83]:
def is_subsequence(seq_small, seq_large):

    for i in range(nlarge - nsmall +1):
        mismatch = False
        for j in range(nsmall):
            if seq_large[i+j] != seq_small[j]:
                mismatch = True
                break

        if not mismatch:
            return True
        
    return False
        
    

In [84]:
is_subsequence(seq_small, seq_large)

True

In [107]:
ser1 = pd.RangeIndex(1, 11).to_series()
ser2 = pd.Series([5, 5, 5, 5, 2, 2, 1, 7, 1, 7, 5, 1, 7, 2])


ordered = ser1[ser1.isin(ser2)].reset_index(drop=True)
ordered

pd.DataFrame(ordered).merge(pd.DataFrame(ser2), how='left').loc[:, 0]

0     1
1     1
2     1
3     2
4     2
5     2
6     5
7     5
8     5
9     5
10    5
11    7
12    7
13    7
Name: 0, dtype: int64

In [1]:
from collections import namedtuple

In [4]:
book = namedtuple('book', ['title', 'num_pages'])
book

__main__.book

In [9]:
s = '{}'.format(1)
s

'1'

In [5]:
d = {'1': 'a'}
tuple(d)

('1',)

In [10]:
from sqlalchemy import Table, Column, Integer, String, MetaData
from sqlalchemy import create_engine

DATABASE = 'books'
engine = create_engine(f'postgresql://postgres:@localhost:5432/{DATABASE}')

meta = MetaData()

In [12]:
meta.drop_all(engine)

In [13]:
from sqlalchemy import Table, Column, Integer, String, MetaData
from sqlalchemy.sql.schema import ForeignKey
from sqlalchemy import create_engine

DATABASE = 'books'
engine = create_engine(f'postgresql://postgres:@localhost:5432/{DATABASE}')
meta = MetaData()

# Create books table
books = Table(
    'books',
    meta,
    Column('id', Integer, primary_key=True),
    Column('title', String),
    Column('num_pages', Integer)
)

# Create authors table
authors = Table(
    'authors',
    meta,
    Column('id', Integer, primary_key=True),
    Column('first_name', String),
    Column('last_name', String)
)



# Create author_books table
author_books = Table(
    'author_books',
    meta,
    Column('id', Integer, primary_key=True),
    Column('author_id', Integer, ForeignKey('authors.id')),
    Column('book_id', Integer, ForeignKey('books.id'))
)

In [29]:
conn = engine.connect()

In [41]:
title = 'Harry Potter 1'
num_pages = 500

In [42]:
r = conn.execute(books.insert().values(title='Harry Potter 1', num_pages=500))

In [43]:
existing_book = conn.execute(
    books.select().where(
        (books.c.title == title) &
        (books.c.num_pages == num_pages)
    ))

In [73]:
existing_book.keys()

RMKeyView(['id', 'title', 'num_pages'])

In [53]:
dict(zip(existing_book.keys(), existing_book.fetchone()))

{'id': 2, 'title': 'Harry Potter 1', 'num_pages': 500}

In [38]:
r.inserted_primary_key[0]

3

In [31]:
conn.execute(books.select()).fetchall()

[(1, 'Harry Potter 1', 500), (2, 'Harry Potter 1', 500)]

In [28]:
conn.execute(books.select().where((books.c.title == 'Harry Potter') & (books.c.num_pages == 500))).fetchone()

In [54]:
conn.close()

In [1]:
from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'postgresql://postgres:@localhost:5432/project_tracker'
app.config['SECRET_KEY'] = '\x9dA$\xbd\x13\xee`\xa8\x9cg\x8c\n\xc3\x04\xa6t^\xfc\x9c\xb3\x13cJ'

db = SQLAlchemy(app)

class Project(db.Model):
    __tablename__ = 'projects'

    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(length=50))

class Task(db.Model):
    __tablename__ = 'tasks'

    id = db.Column(db.Integer, primary_key=True)
    description = db.Column(db.String(length=255))


class ProjectTask(db.Model):
    __tablename__ = 'project_tasks'

    id = db.Column(db.Integer, primary_key=True)
    project_id = db.Column(db.Integer, db.ForeignKey('projects.id'))
    task_id = db.Column(db.Integer, db.ForeignKey('tasks.id'))

/Users/abhi/Documents/coding/advanced-python-working-with-databases/venv/lib/python3.9/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [2]:
p = Project.query.all()[0]
p.title

'Learn Kedro!'

In [3]:
project = Project.query.where(Project.id == 1).first()
tasks = Task.query.where()

In [25]:
existing_project = Project.query.filter(Project.title == "Learn Kedro").first()
existing_project

In [4]:
tasks = Task.query.join(ProjectTask, 
                        (Task.id == ProjectTask.task_id) & 
                        (ProjectTask.project_id == 3)).all()
tasks

[]